In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read the CSV file (Stock_Index Csv used to train the model)
df = pd.read_csv('pred.csv')

# Fill null values in 'Close' column with the last valid observation
df['Close'] = df['Close'].fillna(method='ffill')

# Extract the 'Close' column
data = df['Close'].values.reshape(-1, 1)

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Check for null values in the DataFrame
print(df.isnull().sum())


Date          0
Open         27
High         27
Low          27
Close         0
Adj Close    27
Volume       27
dtype: int64


In [ ]:
def evaluate():
    """
    Sample evaluation function
    Don't modify this function
    Args:
        predict_func (function): The prediction function to use

    """
    df = pd.read_csv('sample_input.csv')
    actual_close = np.loadtxt('sample_close.txt')

    pred_close = predict_func(df)

    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close - pred_close))

    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr) - np.array(pred_prev)
    actual_dir = np.array(actual_curr) - np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir * actual_dir) > 0) * 100

    print(f"Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}")


def predict_func(data):
    """
    Modify this function to predict closing prices for next 2 samples.
    Take care of null values in the sample_input.csv file which are listed as NAN in the dataframe passed to you
    Args:
        data (pandas Dataframe): contains the 50 continuous time series values for a stock index

    Returns:
        list (2 values): your prediction for closing price of next 2 samples
    """
    # Preprocess the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    close_data = data['Close'].fillna(method='ffill')  # Fill null values with the last valid observation
    scaled_data = scaler.fit_transform(close_data.values.reshape(-1, 1))

    # Prepare the input sequences
    sequence_length = 10  # Length of the input sequence
    X = []
    y = []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i - sequence_length:i])
        y.append(scaled_data[i, 0])  # Close column is now the first column after scaling

    X = np.array(X)
    y = np.array(y)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=100, batch_size=32)

    # Prepare the input sequence for prediction
    last_sequence = scaled_data[-sequence_length:]
    next_sequence = []

    # Generate predictions for the next 2 samples
    for _ in range(2):
        prediction = model.predict(last_sequence.reshape(1, sequence_length, X.shape[2]))
        next_sequence.append(prediction[0, 0])
        last_sequence = np.concatenate((last_sequence[1:], prediction.reshape(1, -1)), axis=0)

    # Inverse transform the predictions
    predictions = scaler.inverse_transform(np.array(next_sequence).reshape(-1, 1))

    return predictions.flatten().tolist()



if __name__ == "__main__":
    evaluate()


Epoch 1/100
2/2 [==============================] - 4s 16ms/step - loss: 0.3590
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 0.2711
Epoch 3/100
2/2 [==============================] - 0s 16ms/step - loss: 0.1976
Epoch 4/100
2/2 [==============================] - 0s 16ms/step - loss: 0.1313
Epoch 5/100
2/2 [==============================] - 0s 15ms/step - loss: 0.0733
Epoch 6/100
2/2 [==============================] - 0s 16ms/step - loss: 0.0327
Epoch 7/100
2/2 [==============================] - 0s 15ms/step - loss: 0.0205
Epoch 8/100
2/2 [==============================] - 0s 15ms/step - loss: 0.0387
Epoch 9/100
2/2 [==============================] - 0s 16ms/step - loss: 0.0434
Epoch 10/100
2/2 [==============================] - 0s 16ms/step - loss: 0.0302
Epoch 11/100
2/2 [==============================] - 0s 16ms/step - loss: 0.0207
Epoch 12/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0189
Epoch 13/100
2/2 [==============================]